## Исследование надёжности заёмщиков - кредитный скоринг.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<b>Содержание:</b>
1. [Чтение данных, изучение информации](#step1)
2. [Предобработка данных(приведение к нужному типу, работа с пропусками и дубликатами, лемматизация, категоризация) ](#step2)
3. [Ответы на вопросы бизнеса](#step3)
4. [Общий вывод](#step4)

###  1. Чтение данных и изучение информации. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pymystem3 import Mystem
from collections import Counter
df = pd.read_csv('/datasets/data.csv')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


После прочтения таблицы из файла можно сделать следующие выводы:
1) в таблице содержится 12 столбцов

2) в таблице содержится 21525 строк

3) столбцы содержат данные разных типов: как числовые так и текстовые.


Также видно, что в колонках "days_employed" и "total_income" есть пропущенные данные.
Для получения представления о данных применили метод describe().

### 2. Предобработка данных

Бросается в глаза, что столбец "children" имеет некорректные значения. Проверим количество детей у клиентов, а также переименуем столбец с возрастом клиента для удобства:

In [2]:
print(df['children'].value_counts(dropna = False ))
df.rename(columns={'dob_years': 'client_age'}, inplace=True)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В таблице в столбце с количеством детей есть категория граждан, у которых 20 детей. В данном случае возможно пропущена запятая (2,0), однако, поскольку количество данных записей мало в сравнении с общим числом данных (76 из 21525 строк), то удалим эти строки как выброс. Из этих же соображений удалим 47 строк "дети" со значением "-1":

In [3]:
df = df[(df['children'] != 20) & (df['children'] !=-1)]
print(df['children'].value_counts(dropna = False ))
df = df.reset_index()
df.head()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


,index,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Видим, что количество строк в таблице уменьшилось, пропали некорректные значения в столбце о детях и название столбца с возрастом клиента переименовано. Также обновили индексы строк.

Обработка пропусков: Проверим есть ли в таблице пропущенные занчения NaN и None

In [4]:
print(df.isnull().sum())

index                  0
children               0
days_employed       2162
client_age             0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2162
purpose                0
dtype: int64


Видно, что 2162 человека числятся как никогда не работавшие (days_employed) и не имеющие дохода (total_income), либо данные не заполнены (т.е. имеют нули или NaN значения). Столбцы коррелируют друг с другом, имеют одинаковое количество незаполненных значений, что логично, поскольку если человек не работал, то не имел дохода. 

Есть предположение, что это могут быть те, кто имеет доход от супруга, гос.служащие(или на гос. попечении), пенсионеры, инвлиды или просто люди, не имеющие возможности подтвердить свой доход. Это приблизительно 10% от всех имеющихся строк, что существенно. Следовательно нужно их корректно заполнить. 

Данные занчения можно заполнить медианой, средним значением или нулями. Выдвенем гипотезу, что все эти категории граждан имеют какой-то источник дохода, тем более раз обращались за кредитом. Поэтому выберем медиану(поскольку в столбце days_employed есть данные о невероятно долгом сроке работы, что тоже некорректно и негативно отразится, если заполнять столбец средним занчением) для заполнения и выведем на экран обновленную информацию о количестве нулевых значений в таблице, чтобы проверить, что пропуски удалены. 

In [5]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
print(df.isnull().sum())
df.head()

index               0
children            0
days_employed       0
client_age          0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


,index,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Вывод: Теперь занчений NaN и нулей не осталось. 
Некорректные данные в столбце "days_employed" могли получится по разным причинам, однако, учтем, что эти столбцы не являются целевыми в исследовании, поэтому продолжим анализировать таблицу.

Также поменяем тип данных в этих столбцах на целочисленный, поскольку срок работы и доход это целочисленные переменные, а также они занимают меньше места в памяти при хранении и более читаемы. Используем метод astype т.к. он изменяет тип серии и возвращает новую серию, что необходимо в нашем случае.

In [6]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 13 columns):
index               21402 non-null int64
children            21402 non-null int64
days_employed       21402 non-null int64
client_age          21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


Вывод: Тип данных изменен, столбцов с типом "с плавающей точкой" не осталось.

Обработка дубликатов: Проверим занчения в таблице на наличие дубликатов методом duplicated(). Также проверим отдельно столбцы методом value_counts. Потенциально проблемы могут находиться в текстовых столбцах, особенно заполненных вручную, а также на всякий случай проверим целевые для нашего исследования столбцы. В нашем случае это 'children', 'education', 'family_status', 'gender', 'income_type', 'debt', 'purpose':

In [7]:
print('Children-----------------------------------------')
print(df['children'].value_counts(dropna = False ))
print('Education-----------------------------------------')
print(df['education'].value_counts(dropna = False ))
print('Family_status-----------------------------------------')
print(df['family_status'].value_counts(dropna = False ))
print('Gender-----------------------------------------')
print(df['gender'].value_counts(dropna = False ))
print('Income_type-----------------------------------------')
print(df['income_type'].value_counts(dropna = False ))
print('Debt-----------------------------------------')
print(df['debt'].value_counts(dropna = False ))
print('Purpose-----------------------------------------')
print(df['purpose'].value_counts(dropna = False ))

Children-----------------------------------------
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64
Education-----------------------------------------
среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
Family_status-----------------------------------------
женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64
Gender-----------------------------------------
F      14154
M       7247
XNA        1
Name: ge

Видим, что столбцы Education, Gender и Purpose требуют доработки. В столбце Gender есть одно неопределенное значение. Проверим данные в этой строке: 

In [8]:
df[df['gender']== 'XNA']

,index,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10646,10701,0,-2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Данные в целевых столбцах есть, и они выглядят корректно, значит можно пренебречь неопределенностью в столбце Gender. Проигнорируем эту ошибку, поскольку она не повлияет на исследование, а данные пригодятся. 

Столбец Education содержит дублированные данные написанные в разном регистре. Исправим это:

In [9]:
df['education'] = df['education'].str.lower()
print(df['education'].value_counts(dropna = False ))
df.head()
#df.describe()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64


,index,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


После проведенных преобразований можно искать дубликаты:

In [10]:
print('Duplicates----------------------------------------')
print(df.duplicated().sum())

Duplicates----------------------------------------
0


Видим, что дубликатов в таблице не обнаружено, можно двигаться дальше. Теперь в столбце 'education' ошибок нет, мы привели весь текст к нижнему регистру.
Осталось разобраться со столбцом Purpose, и это уже задача для лемматизации.

### Лемматизация

Для того, чтобы корректно группировать данные необходимо привести слова к их леммам. Выполним это и запишем новые значения в новый столбец таблицы df 'lem_purpose'. Также соединим эти группы слов построчно в одно предложение, разделенное пробелами, для облегчения дальнейшего поиска в строках. Затем выведем этот столбец на экран, сгруппировав значения, а также распечатаем первые 5 строк общей таблицы, чтобы убедиться, что столбец 'lem_purpose' корректно добавлен:

In [11]:
m = Mystem()

res = []
for row in df['purpose']:
    lemmas = ''.join(m.lemmatize(row))
    res.append(lemmas)
df['lem_purpose'] = res
df.head(5)

,index,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose
0,0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n


Проанализируем полученные леммы. Выведем еще раз столбец Цель кредита с количеством запросов:

In [12]:
print(df['lem_purpose'].value_counts(dropna = False ))

автомобиль\n                                968
свадьба\n                                   796
на проведение свадьба\n                     772
сыграть свадьба\n                           769
операция с недвижимость\n                   673
покупка коммерческий недвижимость\n         661
покупка жилье для сдача\n                   651
операция с жилье\n                          648
операция с коммерческий недвижимость\n      646
жилье\n                                     642
покупка жилье\n                             641
покупка жилье для семья\n                   640
недвижимость\n                              632
строительство собственный недвижимость\n    628
операция со свой недвижимость\n             626
строительство жилой недвижимость\n          622
строительство недвижимость\n                620
покупка недвижимость\n                      619
покупка свой жилье\n                        619
ремонт жилье\n                              609
покупка жилой недвижимость\n            

Видим, что все работает. 
Теперь необходимо разделить данные в столбце 'lem_purpose' по категориям. Обратим внимание на количество запросов в каждой из существующих категорий. Оно не имеет выраженных минимумов\максимумов, все запросы распределены в диапазоне от 412 до 968 запросов, а значит достаточное количество данных для анализа. Поэтому при категоризации в перую очередь будем обращать внимание на смысл, описание самой цели. Из сгруппированного выше списка видно, что цели кредита по смыслу можно разделить на следующие категории: 
    
<ul>
  <li>Жилая недвижимость</li>
  <li>Ремонт недвижимости</li>
  <li>Коммерческая недвижимости</li>
  <li>Автомобиль</li>
  <li>Свадьба</li>
  <li>Образование</li>
</ul>
    
Категоризация по группам связанным с автомобилем, свадьбой и образованием очевидна, здесь будем ориентироваться на эти меммы и разнесем запросы по группам.
    
Немного сложнее определиться с запросами имеющими отношение к недвижимости. В категорию Жилая недвижимость, на мой взгляд, можно отнести цели, описанные как "сделки, покупка, операция, жилье, строительство", т.к. все эти цели сообщают о желании клиента реализовать планы с недвижимостью для личных нужд. Отдельно, пожалуй, можно выделить категорию Ремонт жилья, поскольку в случае невыплаты кредита недвижимость под которую берется кредит не останется у банка, тогда получается, что это просто потребительский кредит с другой кредитной ставкой и договором. В отдельную категорию я бы выделила леммы, содержащие слово "коммерческий" относительно недвижимости, по причине того, что это не личные цели, а бизнес, что может сказаться на способности вернуть кредит, может быть сопряжено с определенными рисками и как следствие, также может влиять на кредитную ставку и требует другого юридического оформления кредитного договора. В этом отчете выделим коммерческую недвижимость в отдельную категорию, однако этот вопрос можно уточнить у заказчика и если он не заинтересован в такой детализации и ему удобнее работать с меньшим количеством групп, то объединить с жилой недвижимостью. 

Выполним это в разделе категоризации данных. Для сортировки быберем уникальный для каждой строки идентификатор, чтобы сортировка была корректной.

### Категоризация данных

In [13]:
res1 = []
for query in df['lem_purpose']:
    if 'свадьб' in query:
        res1.append('Свадьба')
    elif ('образован') in query:
        res1.append('Образование')
    elif ('автомоб') in query:
        res1.append('Автомобиль')
    elif ('коммерч') in query:
        res1.append('Коммерческая недвижимость')
    elif ('ремонт') in query:
        res1.append('Ремонт недвижимости')
    elif ('строительст') in query:
        res1.append('Жилая недвижимость')
    else:
        res1.append('Жилая недвижимость')

df['lem_purpose_correct'] = res1
print(df['lem_purpose_correct'].value_counts(dropna = False))

Жилая недвижимость           8864
Автомобиль                   4288
Образование                  3997
Свадьба                      2337
Коммерческая недвижимость    1307
Ремонт недвижимости           609
Name: lem_purpose_correct, dtype: int64


Данные категоризированы, у нас получилось 6 категорий. Теперь можно присвоить значения столбца 'lem_purpose_correct' в первоначальный столбец 'purpose' и удалить лишние столбцы. Также выведем на экран первые 5 значений таблицы, чтобы убедиться, что все сработало корректно.

In [14]:
df['purpose'] = df['lem_purpose_correct']
df = df.drop(['lem_purpose', 'lem_purpose_correct'], axis=1)
df.head()

,index,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,Жилая недвижимость
1,1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,Автомобиль
2,2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,Жилая недвижимость
3,3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,Образование
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,Свадьба


### Вывод

Вывод: данные подготовлены для дальнейшего анализа, теперь можно приступать к ответам на поставленные бизнесом задачи.

### 3. Ответы на вопросы бизнеса:

<b>3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?</b>

Построим наглядную таблицу, чтобы увидеть численные значения и переведем сразу в проценты.

In [15]:
print(df['children'].value_counts())
pd.crosstab(df.children,df.debt, normalize='index').round(4)*100

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


debt,0,1
children,,
0,92.49,7.51
1,90.78,9.22
2,90.56,9.44
3,91.82,8.18
4,90.24,9.76
5,100.00,0.00


### Вывод

Из полученной таблицы видим, что люди, не имеющие детей чаще возвращают кредит в срок, а также обращаются за ним в банк (более половины от всех клиентов). Однако разница с теми у кого дети есть (практически независимо от их количества) несущественна. Они почти также ответственно относятся к погашеню долга. Конечно, бросается в глаза 100% -ное погашение кредита заемщиками с 5-ю детьми, однако выборки слишком мала(всего 9 человек), чтобы говорить о систематисеском явлении. Возможно данная категория получает государственные выплаты на погашение кредита как многодетная семья, и это является гарантом. Поскольку даже в сравнении с семьями с 4-мя детьми ситуция выглядит позитивнее. Возможно 5-й ребенок является пороговым значением для гос. поддержки. 

Суммарно не более 10% клиентов имеют непогашеные в срок кредиты. Вывод - наличие детей не влияет существенно на возврат кредита, однако предпочтение стоит отдать бездетным клиентам и семьям с 5 детьми (для этого утверждения нужна дополнительная статистика).

<b>3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?</b>

In [16]:
print(df['family_status'].value_counts())
pd.crosstab(df.family_status,df.debt, normalize='index').round(4)*100
#pd.crosstab(df.family_status, df.debt, margins = True).style.background_gradient(cmap='autumn_r')

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64


debt,0,1
family_status,,
Не женат / не замужем,90.25,9.75
в разводе,92.94,7.06
вдовец / вдова,93.38,6.62
гражданский брак,90.75,9.25
женат / замужем,92.46,7.54


### Вывод

Здесь также можно сказать, что семейный статус не существенно влияет на возврат долга, однако люди, никогда не состоявшие в официальных отношениях примерно на 2-3% чаще имеют проблемы с оплатой кредита. Следовательно, предпочтение при принятии решения следует отдать тем, кто состоит или когда-либо состоял в официальном браке, и таких клиентов около половины из всех обратившихся. Видимо, это связано с большей готовностью брать на себя ответственность и умением планировать бюджет.

<b>3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</b>

Для ответа на этот вопрос сначала необходимо классифицировать данные в столбце "доход", а затем как в примeрах выше, создать сводную таблицу. Выполним эти манипулции:

In [17]:
kvant = df['total_income']/1000
kvant.value_counts()
kvant.describe()

count    21402.000000
mean       165.182615
std         97.991867
min         20.667000
25%        107.716000
50%        145.020000
75%        195.572250
max       2265.604000
Name: total_income, dtype: float64

In [18]:
def sort_income(data):
    sort = []
    for i in data:
        if 0 < i <= 107700:
            sort.append('низкий')
        elif 107000 < i <= 145000:
            sort.append('средний')
        elif 145000 < i <= 195500:
            sort.append('выше среднего')
        elif 195500 < i :
            sort.append('высокий')
        else:
            sort.append('некорректное значение')
    return(sort)

df['sorted_income'] = sort_income(df['total_income'])
print(df['sorted_income'].value_counts())
pd.crosstab(df.sorted_income,df.debt, normalize='index').round(4)*100
#pd.crosstab(df.total_income, df.debt, margins = True).style.background_gradient(cmap='autumn_r')

выше среднего    6428
высокий          5356
низкий           5346
средний          4272
Name: sorted_income, dtype: int64


debt,0,1
sorted_income,,
высокий,92.83,7.17
выше среднего,91.47,8.53
низкий,92.01,7.99
средний,91.27,8.73


### Вывод

В результате фильтрации и сведения данных в таблицу видим, что люди со средним доходом самая малочисленная категория граждан, обращающихся за кредитом. Налицо логичная закономерность - чем выше доход клиента, тем выше вероятность возврата кредита в срок. Однако люди с низким уровнем дохода всеже надежнее как заемщики, чем люди со средним доходом, хотя этот разброс в размере менее 1% можно считать несущественным. Разброс по всей выборке также невелик и составляет 1,56% Самыми выгодными клиентами в этой классификации являются клиенты с высоким доходом, они чуть более ответственнее других относятся к своим обязательствам, и составляют основную массу клиентов.

Однако риск невозврата\задержки погашения составляет не более 9% для любой категории дохода, а суммарное колебание не более 1,56%.

<b>3.4. Как разные цели кредита влияют на его возврат в срок?</b>

In [19]:
print(df['purpose'].value_counts())
pd.crosstab(df.purpose,df.debt, normalize='index').round(4)*100

Жилая недвижимость           8864
Автомобиль                   4288
Образование                  3997
Свадьба                      2337
Коммерческая недвижимость    1307
Ремонт недвижимости           609
Name: purpose, dtype: int64


debt,0,1
purpose,,
Автомобиль,90.67,9.33
Жилая недвижимость,92.70,7.30
Коммерческая недвижимость,92.50,7.50
Образование,90.77,9.23
Ремонт недвижимости,94.25,5.75
Свадьба,92.17,7.83


### Вывод

Относительно влияния целей кредита на его возврат можно сказать, что самыми частым и при этом самым надежным клиентом в этой категории являются клиенты с целью сделать ремонт в квартире. А самыми не надежными целями являются образование и автомобиль по параметру задолжности, а кроме того, стоит обратить внимание, что это одни из самых распространенных целей. К вопросам связанным с недвижимостью люди подходят более основательно, поэтому при такой цели риск невелик.

Также прослеживается обнаруженная ранее закономерность - не более 10% клиентов независимо от целей имеют трудности с погашением кредита в срок.

### 4. Общий вывод

В целом из исследования видно, что риск невозврата\задержки погашения кредита для любой из рассмотреных зависимостей составляет не более 10%. А разница в референсных группах колеблется в пределах 2-3%.

In [20]:
print('КОНЕЦ ОСНОВНОГО РАЗДЕЛА------------------------------------------------------------------')

КОНЕЦ ОСНОВНОГО РАЗДЕЛА------------------------------------------------------------------


<b>5. Дополнительный раздел, возможные рекомендации.</b>

Есть предположение, что на возврат кредита могут больше влиять другие характеристики заемщика, имеющиеся в таблице. Проверим влияние образования, пола, типа дохода на кредитную историю. Столбец возаст коррелирует с образованием, поэтому детально в этом исследовании его рассматривать не будем.

In [21]:
print(df['gender'].value_counts())
pd.crosstab(df.gender,df.debt, normalize='index').round(4)*100

F      14154
M       7247
XNA        1
Name: gender, dtype: int64


debt,0,1
gender,,
F,93.01,6.99
M,89.75,10.25
XNA,100.00,0.00


In [22]:
print(df['income_type'].value_counts())
pd.crosstab(df.income_type,df.debt, normalize='index').round(4)*100

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


debt,0,1
income_type,,
безработный,50.00,50.00
в декрете,0.00,100.00
госслужащий,94.08,5.92
компаньон,92.60,7.40
пенсионер,94.37,5.63
предприниматель,100.00,0.00
сотрудник,90.46,9.54
студент,100.00,0.00


In [23]:
print(df['education'].value_counts())
pd.crosstab(df.education,df.debt, normalize='index').round(4)*100

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64


debt,0,1
education,,
высшее,94.69,5.31
начальное,89.01,10.99
неоконченное высшее,90.82,9.18
среднее,91.05,8.95
ученая степень,100.00,0.00


<b>Вывод 2:</b>

В итоге видно, что от таких параметров как пол, тип дохода и образования зависимость выше, чем от рассмотренных в основном разделе. Женщины аккуратннее планируют бюджет и платят кредит в срок чаще мужчин. 
в плане занятости предпочтение стоит отдать пенсионерам и госслужащим, однако сотрудники являются самой многочисленной категорией и не выходят за рамки средних 10%, поэтому их стоит оценивать более внимательно по другим характеристикам.
Также стоит обратить внимание на образование соискателя, учитывая, что люди с высшим образованием имеют низкий процент невозврата долга. Люди со средним образованием также хорошие клиенты, поскольку имеют низкий процент невозврата и при этом это самая многочисленная категория. А начальное образование лучше воспринять как негативный фактор.

Таким образом, идеальный заемщик для банка это:
бездетная, замужняя пенсионерка или госслужащая с очень высоким доходом и с ученой степенью, желающая получить кредит на ремонт жилья. Однако в жизни вероятнее встретить иные комбинации.